### Toxicity
Using Sentiment Analysis, determine the top 5 positive subreddits and top 5 negative subreddits based on comment sentiment.
Choose two subreddits focused on similar topics but with different views, e.g., /r/apple and /r/android. Compare the toxicity of the two.

In [25]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType, IntegerType
from  pyspark.sql.functions import input_file_name
from pyspark.sql import SQLContext, Row

df = sqlContext.read.json("hdfs://orion11:32001/sampled_reddit/*")
columns = [
    "distinguished",
    "downs",
    "created_utc",
    "controversiality",
    "edited",
    "gilded",
    "author_flair_css_class",
    "id",
    "author",
    "retrieved_on",
    "score_hidden",
    "subreddit_id",
    "score",
    "name",
    "author_flair_text",
    "link_id",
    "archived",
    "ups",
    "parent_id",
    "subreddit",
    "body"]

#df.show(n=2)
type(df)

pyspark.sql.dataframe.DataFrame

In [26]:
df.createOrReplaceTempView("TEMP_DF")
df = spark.sql("""select first(temp_df.body) text, temp_df.subreddit from TEMP_DF GROUP BY temp_df.subreddit""")
df.show(n=2)

+--------------------+-----------+
|                text|  subreddit|
+--------------------+-----------+
|
# *BOOK..FREE "T...|12monthloan|
|Operation screens...|   160thsor|
+--------------------+-----------+
only showing top 2 rows



In [27]:
#drop empty strings
df = df.dropna()

In [21]:
df.show(n=6)

+--------------------+-----------+
|                text|  subreddit|
+--------------------+-----------+
|you may never kno...|A1B21F8244F|
|Right, I'm not a ...|     ASRoma|
|It's a fascinatin...|     AdPorn|
|    She looks yummy!|    Amateur|
|Made me giggle, t...| ArtHistory|
|               Done!|  Austin512|
+--------------------+-----------+
only showing top 6 rows



In [28]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return str(score)

pand = df.toPandas()
pand['analysis'] = pand['text'].apply(sentiment_analyzer_scores)
pand = pand.drop(columns=['text'])
pand.iloc[:4]

,subreddit,analysis
0,12monthloan,"{'neg': 0.171, 'neu': 0.721, 'pos': 0.108, 'co..."
1,160thsor,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,1970s,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,2QB,"{'neg': 0.044, 'neu': 0.863, 'pos': 0.093, 'co..."


In [29]:
pand.to_csv('sentimental_analysis_sample.csv')